In [1]:
import sys
print(sys.executable)

/Users/saithirumalreddy/anaconda3/bin/python


In [2]:
!pip install wordcloud

In [3]:
import string
import numpy as np
from collections import Counter
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# nltk
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [4]:
restinfo = pd.read_csv("BangaloreZomatoData.csv")

restinfo.head()

,Name,URL,Cuisines,Area,Timing,Full_Address,PhoneNumber,IsHomeDelivery,isTakeaway,isIndoorSeating,isVegOnly,Dinner Ratings,Dinner Reviews,Delivery Ratings,Delivery Reviews,KnownFor,PopularDishes,PeopleKnownFor,AverageCost
0,Sri Udupi Park,https://www.zomato.com/bangalore/sri-udupi-par...,"South Indian, North Indian, Chinese, Street Fo...","Indiranagar, Bangalore",7am – 11pm (Today),"273, Monalisa, 6th Main, 100 Feet Road, Indira...",+919945977774,1,1,1,1,4.0,462,4.1,16000,NaN,"Filtered Coffee, Sambhar, Pav Bhaji, Gobi Manc...","Economical, Prompt Service, Hygiene, Quality F...",450
1,Meghana Foods,https://www.zomato.com/bangalore/meghana-foods...,"Biryani, Andhra, North Indian, Seafood","Indiranagar, Bangalore",Opens at 6:30pm,"544, First Floor, CMH Road, Near Indiranagar M...",+918041135050,1,1,1,0,4.3,1654,4.3,28600,Spicy Chicken Biryani,"Authentic Hyderabadi Biryani, Paneer Biryani, ...","Boneless Chicken Biryani, Ample Seating Area, ...",700
2,Donne Biriyani House,https://www.zomato.com/bangalore/donne-biriyan...,Biryani,"Indiranagar, Bangalore",11am – 11pm (Today),"8/ 9, 17th F Cross, 2nd Stage, Indiranagar, Ba...",+918861564169,1,1,1,0,3.9,411,3.5,33200,NaN,NaN,"Great Recommendations, Nice Taste, Great Ambia...",300
3,Domino's Pizza,https://www.zomato.com/bangalore/dominos-pizza...,"Pizza, Fast Food, Desserts","Indiranagar, Bangalore",10:57am – 12midnight (Today),"308, 2nd Stage, 100 Feet Road, Indiranagar, Ba...",+919916465787,1,1,1,0,2.4,422,4.4,8205,NaN,"Barbeque Chicken Pizza, Choco Lava Cake, White...","Value for Money, Packaging, Staff, Ambience, Food",400
4,KFC,https://www.zomato.com/bangalore/kfc-indiranagar,"Burger, Fast Food, Biryani, Desserts, Beverages","Indiranagar, Bangalore",11am – 11pm (Today),"38/1A, CMH Road, Indiranagar, Bangalore",+919513700040,1,1,1,0,2.8,673,4.0,9148,NaN,"Fiery Chicken, Chicken Popcorn, Rice Bowl, Wings","Elegantly Decorated, Great Recommendations, Vi...",400


In [5]:
restinfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8923 entries, 0 to 8922
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              8923 non-null   object
 1   URL               8923 non-null   object
 2   Cuisines          8923 non-null   object
 3   Area              8923 non-null   object
 4   Timing            5820 non-null   object
 5   Full_Address      8923 non-null   object
 6   PhoneNumber       8923 non-null   object
 7   IsHomeDelivery    8923 non-null   int64 
 8   isTakeaway        8923 non-null   int64 
 9   isIndoorSeating   8923 non-null   int64 
 10  isVegOnly         8923 non-null   int64 
 11  Dinner Ratings    8923 non-null   object
 12  Dinner Reviews    8923 non-null   int64 
 13  Delivery Ratings  8923 non-null   object
 14  Delivery Reviews  8923 non-null   int64 
 15  KnownFor          258 non-null    object
 16  PopularDishes     1535 non-null   object
 17  PeopleKnownFor

In [6]:
restinfo=restinfo[['Name','Cuisines','Area','PhoneNumber','IsHomeDelivery','isTakeaway','isIndoorSeating','isVegOnly','AverageCost']]

In [7]:
restinfo.head()

,Name,Cuisines,Area,PhoneNumber,IsHomeDelivery,isTakeaway,isIndoorSeating,isVegOnly,AverageCost
0,Sri Udupi Park,"South Indian, North Indian, Chinese, Street Fo...","Indiranagar, Bangalore",+919945977774,1,1,1,1,450
1,Meghana Foods,"Biryani, Andhra, North Indian, Seafood","Indiranagar, Bangalore",+918041135050,1,1,1,0,700
2,Donne Biriyani House,Biryani,"Indiranagar, Bangalore",+918861564169,1,1,1,0,300
3,Domino's Pizza,"Pizza, Fast Food, Desserts","Indiranagar, Bangalore",+919916465787,1,1,1,0,400
4,KFC,"Burger, Fast Food, Biryani, Desserts, Beverages","Indiranagar, Bangalore",+919513700040,1,1,1,0,400


In [8]:
restinfo.isnull().sum()

Name               0
Cuisines           0
Area               0
PhoneNumber        0
IsHomeDelivery     0
isTakeaway         0
isIndoorSeating    0
isVegOnly          0
AverageCost        0
dtype: int64

In [9]:
restinfo.dropna(inplace=True)

In [10]:
restinfo.iloc[0].Cuisines

'South Indian, North Indian, Chinese, Street Food, Biryani, Ice Cream, Beverages, Shake'

In [11]:
restinfo['Cuisines'] = restinfo['Cuisines'].str.split(', ')
restinfo['Area'] = restinfo['Area'].str.split(', ')

In [12]:
restinfo.head()

,Name,Cuisines,Area,PhoneNumber,IsHomeDelivery,isTakeaway,isIndoorSeating,isVegOnly,AverageCost
0,Sri Udupi Park,"[South Indian, North Indian, Chinese, Street F...","[Indiranagar, Bangalore]",+919945977774,1,1,1,1,450
1,Meghana Foods,"[Biryani, Andhra, North Indian, Seafood]","[Indiranagar, Bangalore]",+918041135050,1,1,1,0,700
2,Donne Biriyani House,[Biryani],"[Indiranagar, Bangalore]",+918861564169,1,1,1,0,300
3,Domino's Pizza,"[Pizza, Fast Food, Desserts]","[Indiranagar, Bangalore]",+919916465787,1,1,1,0,400
4,KFC,"[Burger, Fast Food, Biryani, Desserts, Beverages]","[Indiranagar, Bangalore]",+919513700040,1,1,1,0,400


In [13]:
restinfo['Cuisines']=restinfo['Cuisines'].apply(lambda x:[i.replace(" ","") for i in x])
restinfo['Area']=restinfo['Area'].apply(lambda x:[i.replace(" ","") for i in x])

In [14]:
restinfo.head()

,Name,Cuisines,Area,PhoneNumber,IsHomeDelivery,isTakeaway,isIndoorSeating,isVegOnly,AverageCost
0,Sri Udupi Park,"[SouthIndian, NorthIndian, Chinese, StreetFood...","[Indiranagar, Bangalore]",+919945977774,1,1,1,1,450
1,Meghana Foods,"[Biryani, Andhra, NorthIndian, Seafood]","[Indiranagar, Bangalore]",+918041135050,1,1,1,0,700
2,Donne Biriyani House,[Biryani],"[Indiranagar, Bangalore]",+918861564169,1,1,1,0,300
3,Domino's Pizza,"[Pizza, FastFood, Desserts]","[Indiranagar, Bangalore]",+919916465787,1,1,1,0,400
4,KFC,"[Burger, FastFood, Biryani, Desserts, Beverages]","[Indiranagar, Bangalore]",+919513700040,1,1,1,0,400


In [15]:
restinfo['PriceRange'] = restinfo['AverageCost'].apply(lambda x: 'cheap' if x < 500 else 'medium' if 500 <= x <= 1500 else 'expensive')

In [16]:
restinfo['IsHomeDelivery'] = restinfo['IsHomeDelivery'].replace({1: 'HomeDeliveryisAvailable', 0: 'HomeDeliveryisNotAvailable'})
restinfo['isTakeaway'] = restinfo['isTakeaway'].replace({1: 'TakeawayisAvailable', 0: 'TakeawayisNotAvailable'})
restinfo['isIndoorSeating'] = restinfo['isIndoorSeating'].replace({1: 'HasIndoorSeating', 0: 'DoesnothaveIndoorSeating'})
restinfo['isVegOnly'] = restinfo['isVegOnly'].replace({1: 'OnlyVeg', 0: 'VegandNonVeg'})

In [17]:
restinfo.head()

,Name,Cuisines,Area,PhoneNumber,IsHomeDelivery,isTakeaway,isIndoorSeating,isVegOnly,AverageCost,PriceRange
0,Sri Udupi Park,"[SouthIndian, NorthIndian, Chinese, StreetFood...","[Indiranagar, Bangalore]",+919945977774,HomeDeliveryisAvailable,TakeawayisAvailable,HasIndoorSeating,OnlyVeg,450,cheap
1,Meghana Foods,"[Biryani, Andhra, NorthIndian, Seafood]","[Indiranagar, Bangalore]",+918041135050,HomeDeliveryisAvailable,TakeawayisAvailable,HasIndoorSeating,VegandNonVeg,700,medium
2,Donne Biriyani House,[Biryani],"[Indiranagar, Bangalore]",+918861564169,HomeDeliveryisAvailable,TakeawayisAvailable,HasIndoorSeating,VegandNonVeg,300,cheap
3,Domino's Pizza,"[Pizza, FastFood, Desserts]","[Indiranagar, Bangalore]",+919916465787,HomeDeliveryisAvailable,TakeawayisAvailable,HasIndoorSeating,VegandNonVeg,400,cheap
4,KFC,"[Burger, FastFood, Biryani, Desserts, Beverages]","[Indiranagar, Bangalore]",+919513700040,HomeDeliveryisAvailable,TakeawayisAvailable,HasIndoorSeating,VegandNonVeg,400,cheap


In [18]:
restinfo['IsHomeDelivery'] = restinfo['IsHomeDelivery'].str.split(', ')
restinfo['isTakeaway'] = restinfo['isTakeaway'].str.split(', ')
restinfo['isIndoorSeating'] = restinfo['isIndoorSeating'].str.split(', ')
restinfo['isVegOnly'] = restinfo['isVegOnly'].str.split(', ')
restinfo['PriceRange'] = restinfo['PriceRange'].str.split(', ')

In [19]:
restinfo.head()

,Name,Cuisines,Area,PhoneNumber,IsHomeDelivery,isTakeaway,isIndoorSeating,isVegOnly,AverageCost,PriceRange
0,Sri Udupi Park,"[SouthIndian, NorthIndian, Chinese, StreetFood...","[Indiranagar, Bangalore]",+919945977774,[HomeDeliveryisAvailable],[TakeawayisAvailable],[HasIndoorSeating],[OnlyVeg],450,[cheap]
1,Meghana Foods,"[Biryani, Andhra, NorthIndian, Seafood]","[Indiranagar, Bangalore]",+918041135050,[HomeDeliveryisAvailable],[TakeawayisAvailable],[HasIndoorSeating],[VegandNonVeg],700,[medium]
2,Donne Biriyani House,[Biryani],"[Indiranagar, Bangalore]",+918861564169,[HomeDeliveryisAvailable],[TakeawayisAvailable],[HasIndoorSeating],[VegandNonVeg],300,[cheap]
3,Domino's Pizza,"[Pizza, FastFood, Desserts]","[Indiranagar, Bangalore]",+919916465787,[HomeDeliveryisAvailable],[TakeawayisAvailable],[HasIndoorSeating],[VegandNonVeg],400,[cheap]
4,KFC,"[Burger, FastFood, Biryani, Desserts, Beverages]","[Indiranagar, Bangalore]",+919513700040,[HomeDeliveryisAvailable],[TakeawayisAvailable],[HasIndoorSeating],[VegandNonVeg],400,[cheap]


In [20]:
restinfo['tags']= restinfo['Cuisines']+restinfo['IsHomeDelivery']+restinfo['isTakeaway']+restinfo['isIndoorSeating']+restinfo['isVegOnly']+restinfo['PriceRange']

In [21]:
df=restinfo[['Name','tags','PhoneNumber']]
df.head()

,Name,tags,PhoneNumber
0,Sri Udupi Park,"[SouthIndian, NorthIndian, Chinese, StreetFood...",+919945977774
1,Meghana Foods,"[Biryani, Andhra, NorthIndian, Seafood, HomeDe...",+918041135050
2,Donne Biriyani House,"[Biryani, HomeDeliveryisAvailable, TakeawayisA...",+918861564169
3,Domino's Pizza,"[Pizza, FastFood, Desserts, HomeDeliveryisAvai...",+919916465787
4,KFC,"[Burger, FastFood, Biryani, Desserts, Beverage...",+919513700040


In [22]:
df['tags']=df['tags'].apply(lambda x:" ".join(x))
df.head()

,Name,tags,PhoneNumber
0,Sri Udupi Park,SouthIndian NorthIndian Chinese StreetFood Bir...,+919945977774
1,Meghana Foods,Biryani Andhra NorthIndian Seafood HomeDeliver...,+918041135050
2,Donne Biriyani House,Biryani HomeDeliveryisAvailable TakeawayisAvai...,+918861564169
3,Domino's Pizza,Pizza FastFood Desserts HomeDeliveryisAvailabl...,+919916465787
4,KFC,Burger FastFood Biryani Desserts Beverages Hom...,+919513700040


In [23]:
!pip install nltk

In [24]:
import nltk

In [25]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [26]:
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [27]:
df['tags'].apply(stem)

0       southindian northindian chines streetfood biry...
1       biryani andhra northindian seafood homedeliver...
2       biryani homedeliveryisavail takeawayisavail ha...
3       pizza fastfood dessert homedeliveryisavail tak...
4       burger fastfood biryani dessert beverag homede...
                              ...                        
8918    fastfood homedeliveryisavail takeawayisavail h...
8919    seafood mangalorean beverag homedeliveryisavai...
8920    salad chines homedeliveryisavail takeawayisava...
8921    thai chines homedeliveryisavail takeawayisavai...
8922    biryani fastfood homedeliveryisavail takeawayi...
Name: tags, Length: 8923, dtype: object

In [28]:
df['tags']=df['tags'].apply(lambda x:x.lower())

In [29]:
df['tags'][0]

'southindian northindian chinese streetfood biryani icecream beverages shake homedeliveryisavailable takeawayisavailable hasindoorseating onlyveg cheap'

In [30]:
df.head()

,Name,tags,PhoneNumber
0,Sri Udupi Park,southindian northindian chinese streetfood bir...,+919945977774
1,Meghana Foods,biryani andhra northindian seafood homedeliver...,+918041135050
2,Donne Biriyani House,biryani homedeliveryisavailable takeawayisavai...,+918861564169
3,Domino's Pizza,pizza fastfood desserts homedeliveryisavailabl...,+919916465787
4,KFC,burger fastfood biryani desserts beverages hom...,+919513700040


In [31]:
cv=CountVectorizer(max_features=1000,stop_words='english')

In [69]:
vectors=cv.fit_transform(df['tags']).toarray()

In [70]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [71]:
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
similarity=cosine_similarity(vectors)

In [73]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1468, 1.0000000000000004),
 (8015, 1.0000000000000004),
 (4674, 0.960768922830523),
 (5733, 0.9230769230769235),
 (735, 0.9198662110078001)]

In [77]:
def recommend(tags):
    tags = tags.lower()
    query_vector = cv.transform([tags]).toarray()
    similarity_scores = cosine_similarity(query_vector, vectors)
    restaurant_list = sorted(list(enumerate(similarity_scores[0])), reverse=True, key=lambda x: x[1])[1:6]

    for i in restaurant_list:
        print(df.iloc[i[0]]['Name'])

SyntaxError: duplicate argument 'tags' in function definition (2887157551.py, line 1)

In [81]:
recommend('southindian homedeliveryisavailable biryani')

Thalaivars Biryani
Donne Biriyanis Junction
Village Donne Biryani
Donne Biryani Corner
Rashmi Food Hub
